Author: Nina Tubau Ribera

In [ ]:
!pip install imagecodecs

In [4]:
import tifffile
import re
import imagecodecs
import os

# Investigation: load data + metadata checks 

In [5]:
image = tifffile.imread('/data/data_drive/DATA/QuPath/Bianca/IHCdata/first.tif')

TiffPages: invalid page offset 19051315216


In [75]:
with tifffile.TiffFile('CD8_Ki67_CW20_x02_Default_Extended.tif') as tif:
    tag_0 = tif.pages[0]

In [49]:
with tifffile.TiffFile('/data/data_drive/DATA_acquired/QuPath/dataShirley/ER-4 TC_Scan1.qptiff') as tif:
    tag = tif.pages[0]

In [50]:
print(tag.tags)

TiffTag 254 NewSubfileType @24 LONG @36 UNDEFINED
TiffTag 256 ImageWidth @44 SHORT @56 32640
TiffTag 257 ImageLength @64 SHORT @76 48960
TiffTag 258 BitsPerSample @84 SHORT @96 8
TiffTag 259 Compression @104 SHORT @116 LZW
TiffTag 262 PhotometricInterpretation @124 SHORT @136 MINISBLACK
TiffTag 270 ImageDescription @144 ASCII[60903] @98796 <?xml version="1.0" encod
TiffTag 277 SamplesPerPixel @164 SHORT @176 1
TiffTag 282 XResolution @184 RATIONAL @196 (4294967295, 213133)
TiffTag 283 YResolution @204 RATIONAL @216 (4294967295, 213133)
TiffTag 284 PlanarConfiguration @224 SHORT @236 CONTIG
TiffTag 286 XPosition @244 RATIONAL @256 (2547490303, 4294967295)
TiffTag 287 YPosition @264 RATIONAL @276 (4294967295, 1273833449)
TiffTag 296 ResolutionUnit @284 SHORT @296 CENTIMETER
TiffTag 305 Software @304 ASCII[16] @159700 PerkinElmer-QPI
TiffTag 306 DateTime @324 ASCII[20] @159716 2020:10:06 02:02:29
TiffTag 322 TileWidth @344 SHORT @356 512
TiffTag 323 TileLength @364 SHORT @376 512
TiffTag 

Regex for Pixel size in X and Y

In [5]:
with tifffile.TiffFile('CD8_Ki67_CW20_x02_Default_Extended.tif') as tif:
    tag = tif.pages[0]
regex = re.compile('PixelSize[X-Y] = ([0-9].[0-9]\w+)')
res = regex.findall(tag.tags['ImageDescription'].value)

# Read .tif and write ome.tif pyramid file  

- Navigating through all folders containing annotation (output CaseViewer converter)
- Read tiff file and save as ome.tif pyramid file (res 1,4 and 16) 
- Save in same directory

Check files in Annotation folders from CaseViewer

In [40]:
for root, dirs, files in os.walk(path):
    if 'Annotation' in root:
        for file in files:
            if file.endswith(".tif"):
                 print(os.path.join(root, file))

X:/Bianca for Nina/KConFab 10.007.0368 BRCA2\2M08_06.10.2021_22.07.24 - Annotation 1\2M08_06.10.2021_22.07.24_Default_Extended.tif
X:/Bianca for Nina/KConFab 10.007.0368 BRCA2\2M08_06.10.2021_22.07.24 - Annotation 1\2M08_06.10.2021_22.07.24_Default_Extended_2021-10-12T10-10-18.458.tif
X:/Bianca for Nina/KConFab 10.007.0368 BRCA2\2M08_06.10.2021_22.07.24 - Annotation 1\2M08_06.10.2021_22.07.24_Default_Extended_2021-10-12T10-09-59.801.tif
X:/Bianca for Nina/KConFab 10.007.0368 BRCA2\2M08_06.10.2021_22.07.24 - Annotation 1\2M08_06.10.2021_22.07.24_Default_Extended_2021-10-12T10-10-49.772.tif
X:/Bianca for Nina/20PM0149 30years Premenopause Parous WT\2M24_07.10.2021_05.29.19 - Annotation 1\2M24_07.10.2021_05.29.19_Default_Extended_2021-10-12T08-41-49.285.tif
X:/Bianca for Nina/20PM0149 30years Premenopause Parous WT\2M24_07.10.2021_05.29.19 - Annotation 1\2M24_07.10.2021_05.29.19_Default_Extended_2021-10-12T08-39-04.709.tif
X:/Bianca for Nina/20PM0149 30years Premenopause Parous WT\2M24_07

In [71]:
for root, dirs, files in os.walk(path):
    if 'Annotation' in root:
        for file in files:
            if file.endswith(".tif"):
                root = root.replace("\\","/")
                to_convert = root + '/'+file
                print(to_convert)
                with tifffile.TiffFile(to_convert) as tif:
                    tag = tif.pages[0]
                #image = tifffile.imread(to_convert)
                regex = re.compile('PixelSize[X-Y] = ([0-9].[0-9]\w+)')
                #res = regex.findall(tag.tags['ImageDescription'].value)
                #print(res[0],res[1])
                if not os.path.isfile(root+'/'+file[:-4]+'temp_converted.ome.tif') and file.endswith('Extended.tif'):
                    with tifffile.TiffWriter(root+'/'+file[:-4]+'temp_converted.ome.tif', bigtiff=True) as tif_:

                        options = dict(tile=(256, 256), compression='jpeg2000')
                        tif_.write(image, subifds=2, **options)
                            # save pyramid levels to the two subifds
                                #tif_.write(image, subfiletype=1, **options)
                        tif_.write(image[::4, ::4], subfiletype=1, **options)
                        tif_.write(image[::16, ::16], subfiletype=1, **options)
                        ##tif_.write(image[::64, ::64], subfiletype=1, **options)



X:/Bianca for Nina/KConFab 10.007.0368 BRCA2/2M08_06.10.2021_22.07.24 - Annotation 1/2M08_06.10.2021_22.07.24_Default_Extended.tif
X:/Bianca for Nina/KConFab 10.007.0368 BRCA2/2M08_06.10.2021_22.07.24 - Annotation 1/2M08_06.10.2021_22.07.24_Default_Extended_2021-10-12T10-10-18.458.tif
X:/Bianca for Nina/KConFab 10.007.0368 BRCA2/2M08_06.10.2021_22.07.24 - Annotation 1/2M08_06.10.2021_22.07.24_Default_Extended_2021-10-12T10-09-59.801.tif
X:/Bianca for Nina/KConFab 10.007.0368 BRCA2/2M08_06.10.2021_22.07.24 - Annotation 1/2M08_06.10.2021_22.07.24_Default_Extended_2021-10-12T10-10-49.772.tif
X:/Bianca for Nina/KConFab 10.007.0368 BRCA2/2M08_06.10.2021_22.07.24 - Annotation 1/2M08_06.10.2021_22.07.24_Default_Extendedtemp_converted.ome.tif
X:/Bianca for Nina/20PM0149 30years Premenopause Parous WT/2M24_07.10.2021_05.29.19 - Annotation 1/2M24_07.10.2021_05.29.19_Default_Extended_2021-10-12T08-41-49.285.tif
X:/Bianca for Nina/20PM0149 30years Premenopause Parous WT/2M24_07.10.2021_05.29.19 - 